# Deep Functional Maps
----

Predict correspondences within similar 3D shapes by using *geometric deep learning* (deep learning for non-Euclidean space). This was published in ICCV 2017 by Litany *et al*., [Deep Functional Maps: Structured Prediction for Dense Shape Correspondence](http://openaccess.thecvf.com/content_iccv_2017/html/Litany_Deep_Functional_Maps_ICCV_2017_paper.html).

![The network architecture](https://raw.githubusercontent.com/orlitany/DeepFunctionalMaps/master/fmnet.png)

This is my attempt trying to understand the paper by reproducing it in this notebook from the source code provided by the first author: [@orlitany](https://github.com/orlitany/DeepFunctionalMaps)

This notebook shows only the prediction process. 

In [1]:
import tensorflow as tf
import numpy as np
import os
import scipy.io as sio
import time

print(tf.__version__)

1.10.1


## Load the input pair data

Load the pair of data. The data is 3D mesh objects which contains values from SHOT descriptor method. These descriptors have been pre-calculated.

The pair is shown below. Left: source. Right: target.

![Test Pair](./images/test_pair.png)

You need to download the data from the link given in [@orlitany](https://github.com/orlitany/DeepFunctionalMaps)

In [2]:
# load the pair of data
source_file = r'./files/tr_reg_080.mat'
target_file = r'./files/tr_reg_081.mat'

source_data = sio.loadmat(source_file)
target_data = sio.loadmat(target_file)

print(source_data.keys())
print(target_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'model_S', 'model_evecs', 'model_evecs_trans', 'model_shot', 'shot_params'])
dict_keys(['__header__', '__version__', '__globals__', 'model_S', 'model_evecs', 'model_evecs_trans', 'model_shot', 'shot_params'])


In [3]:
# I don't know exactly the meaning of each field in the data, but this is what the authors coded to prepare the input data
# What I can figure out is that part == source and model == target

input_data = {}
input_data.update(source_data)

input_data['part_evecs'] = input_data['model_evecs']
input_data.pop('model_evecs')

input_data['part_evecs_trans'] = input_data['model_evecs_trans']
input_data.pop('model_evecs_trans')

input_data['part_shot'] = input_data['model_shot']
input_data.pop('model_shot')

input_data.update(target_data)

## Load the network model

The trained model is saved in `log/train_inter_k_flag/model.ckpt-1270` file.

In [5]:
log_dir = './files/log/train_inter_k_flag'
meta_file = os.path.join(log_dir, 'model.ckpt-1270.meta')

# read the stored graph
sess = tf.Session()
saver = tf.train.import_meta_graph(meta_file)
saver.restore(sess, tf.train.latest_checkpoint(log_dir))
graph = tf.get_default_graph()

INFO:tensorflow:Restoring parameters from ./files/log/train_inter_k_flag/model.ckpt-1270


In [6]:
# retrieve placeholder variable names at the input layer, used when we feed the network
part_shot = graph.get_tensor_by_name('part_shot:0')
model_shot = graph.get_tensor_by_name('model_shot:0')
dist_map = graph.get_tensor_by_name('dist_map:0')
part_evecs = graph.get_tensor_by_name('part_evecs:0')
part_evecs_trans = graph.get_tensor_by_name('part_evecs_trans:0')
model_evecs = graph.get_tensor_by_name('model_evecs:0')
model_evecs_trans = graph.get_tensor_by_name('model_evecs_trans:0')
phase = graph.get_tensor_by_name('phase:0')

In [7]:
# retrieve variable names for the output
Ct_est = graph.get_tensor_by_name('MatrixSolveLs:0')
softCorr = graph.get_tensor_by_name('pointwise_corr_loss/soft_correspondences:0')

## Run the test

Create feed dictionary for the network model

In [8]:
num_evecs= 120

feed_dict = {phase: True,
             part_shot: [input_data['part_shot']],
             model_shot: [input_data['model_shot']],
             dist_map: [[[None]]],
             part_evecs: [input_data['part_evecs'][:, 0:num_evecs]],
             part_evecs_trans: [input_data['part_evecs_trans'][0:num_evecs, :]],
             model_evecs: [input_data['model_evecs'][:, 0:num_evecs]],
             model_evecs_trans: [input_data['model_evecs_trans'][0:num_evecs, :]]
            }

In [9]:
# run the test
t = time.time()
Ct_est_out, softCorr_out  = sess.run([Ct_est, softCorr], feed_dict=feed_dict)
C_est_out = Ct_est_out.transpose([0, 2, 1])
print('Computed correspondences for pair: {}, {}. Took {} seconds'.format('tr_reg_080', 'tr_reg_081', time.time() - t))
sess.close()

Computed correspondences for pair: tr_reg_080, tr_reg_081. Took 2.5087170600891113 seconds


In [10]:
# save the result
out = {
    'C_est': C_est_out,
    'softCorr': softCorr_out
}
sio.savemat(r'./files/pred/test_result.mat', out)

## Visualize the test result

Due to 3D limitation of jupyter, visualization was done in Matlab (see author's implementation). The colormap of the target (right) was taken by the maximum response of the `softCorr` indexed by the colormap of the source (left).

![Test Pair Result](./images/test_pair_result.png)

```
# get the maximum response
[~, matches] = max(softCorr, [], 1);

# create source colormap that is unique for each vertex
# read the tr_reg_080.off file that contains the vertices and faces 
# assume that V = Nx3 vertices and F = Nx3 triangle faces
mins = min(V,[],1);
maxs = max(V,[],1);
npts = size(V,1)
source_cmap = (V - ones(npts, 1)*mins) ./ (ones(npts, 1) * (maxs-mins))

# get the indexed colormap
target_cmap = source_cmap(matches,:);
```